<a href="https://colab.research.google.com/github/Anshuman-37/Summer_Dissertation/blob/main/Registered_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### FSL Installation and Kaggle Prerequisties

In [1]:
# ## On Average Takes around 16 minutes 
import os
!sudo apt -qq install file
!wget https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py
!python2 fslinstaller.py

fslpath = "/usr/local/fsl"
os.environ["FSLDIR"] = fslpath
os.environ["PATH"] += os.pathsep + os.path.join(fslpath, 'bin')
!. ${FSLDIR}/etc/fslconf/fsl.sh

!flirt -version 

Checking FSL package
[OK] File downloaded
Installing FSL software version 6.0.5.2...
-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|[OK] FSL software installed.
/-Performing post install tasks
\Stage 1
By installing this python distribution you agree to the license terms in
/usr/local/fsl/fslpython/LICENSE.txt
100%
S

In [3]:
## Upload your kaggle.json file that can be downloaded from kaggle account
## Takes a bit time to read the kaggle.json file Run twice if it didn't found it
!pip install kaggle --upgrade
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!mkdir content/data
!kaggle datasets download -d lyfeisgood/something-bigger
!unzip /content/something-bigger.zip -d data_new
!pip install regtricks
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘content/data’: No such file or directory
100% 2.12G/2.12G [00:21<00:00, 125MB/s]
100% 2.12G/2.12G [00:21<00:00, 107MB/s]
Archive:  /content/something-bigger.zip
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/aCBV.nii  
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/aCBV_var.nii  
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/arrival.nii  
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/arrival_var.nii  
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/mask.nii  
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/modelfit_mean.nii  
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/perfusion.nii  
  inflating: data_new/Main_Data/sub-1269766/calib_voxelwise/perfusion_var.nii  
  inflating: data_new/Main_Data/sub-1269766/data2std/T1_2_std.nii  
  inflating: data_new/Main_Data/sub

#### Header Files

In [4]:
### Common Libraries
import os
import re
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

### Misc. 
import regtricks as rt

### Torch Files 
import torch
print(torch.__version__)
print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from torchsummary import summary

## Import my .py file ....... 
import Data_Loader as dl
import Models as model
import Prediction as pd


1.12.1+cu113
True
1.12.1+cu113
True
cuda:0
1.12.1+cu113
True


#### 2D convolution

##### Data Loading

In [5]:
# Take around 10 minutes
## Defining the path of the data 
x,y = dl.data_loader_2D('/content/data_new','/content/data_new',55,device)

## Printing the shape of specifid data
print('\nShape of MRI images - >');     dl.print_data_shape(x); 
print('\nShape of ASL images - >');     dl.print_data_shape(y);

## Printing the Stats of the data
print('\nStats for MRI data(X) - >');   dl.tensor_stats(x); 
print('\nStats for ASL data(X) - >');   dl.tensor_stats(y);

## Printing the Dimension of X and Y  
## Dimension refer -> Number x Length x Breadth x Height x Channel
print('\nDimensions of X(MRI Data) ->',end=' '); dl.print_data_dimension(x)
print('\nDimensions of Y(ASL Data) ->',end=' '); dl.print_data_dimension(y)

train_x , train_y , test_x , test_y = dl.data_split(x,y,0.2)

print('\nDimensions of Train Test Data (MRI) ->',end=' ');      dl.print_data_dimension(train_x) ; dl.print_data_dimension(test_x);
print('\nDimensions of Train Test Data (ASL Data) ->',end=' '); dl.print_data_dimension(train_y) ; dl.print_data_dimension(test_y);

torch.Size([23, 1, 91, 109, 91])
torch.Size([23, 1, 91, 109, 91])

Shape of MRI images - >
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])

Shape of ASL images - >
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Size([1, 91, 109])
torch.Siz

##### Model Training

In [ ]:
## Training the model
train_loss, test_loss, predictions = model.train_conv_2D(2500,4,train_x,train_y,test_x,test_y,50)
print('Train Loss',train_loss); print('Test Loss ',test_loss); print('Predictions shape', predictions.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 200, 91, 109]           2,000
       BatchNorm2d-2         [-1, 200, 91, 109]             400
              ReLU-3         [-1, 200, 91, 109]               0
         MaxPool2d-4         [-1, 200, 91, 109]               0
            Conv2d-5         [-1, 400, 91, 109]         720,400
       BatchNorm2d-6         [-1, 400, 91, 109]             800
              ReLU-7         [-1, 400, 91, 109]               0
         MaxPool2d-8         [-1, 400, 91, 109]               0
            Conv2d-9         [-1, 800, 91, 109]       2,880,800
      BatchNorm2d-10         [-1, 800, 91, 109]           1,600
             ReLU-11         [-1, 800, 91, 109]               0
        MaxPool2d-12         [-1, 800, 91, 109]               0
           Conv2d-13         [-1, 400, 91, 109]       2,880,400
      BatchNorm2d-14         [-1, 400, 

  0%|          | 1/2501 [00:01<42:45,  1.03s/it]

0.19734068
2553.4814453125


  1%|          | 26/2501 [00:26<42:07,  1.02s/it]

0.012958062
2553.4814453125


...

0.00046517077
2553.4814453125


 67%|██████▋   | 1676/2501 [28:34<14:03,  1.02s/it]

0.00053761114
2553.4814453125


 68%|██████▊   | 1701/2501 [28:59<13:38,  1.02s/it]

0.00035663357
2553.4814453125


##### Results and Plots

In [ ]:
#pd.predictions_2D(test_y,predictions,3)
def plot_residuals_2D(y_target,y_predicted,sample_number):
    y_p = y_predicted[sample_number]
    y = y_target.cpu().detach().numpy()
    y_p = y_p.cpu().detach().numpy()
    # print(y_p.shape); print(type(y_p))
    # print(y.shape); print(type(y))

    plt.imshow((y[sample_number][0,:,:]-y_p[0,:,:])**2,cmap = 'gray'); plt.show();
plot_residuals_2D(test_y,predictions,3)
pd.predictions_2D(test_y,predictions,3)


#### MISC

In [27]:
print((torch.cuda.memory_allocated())/(1024*1024));
!nvidia-smi

3616.96728515625
Sun Aug 14 21:14:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    39W / 250W |   8329MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                       

In [24]:
torch.cuda.empty_cache()
# dl.print_data_shape(predictions[0])

In [ ]:
# x_first = predictions[0][0]
# pred_np = x_first.cpu().detach().numpy()
# dl.print_data_dimension(pred_np); print(type(pred_np))

# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import matplotlib.animation as animation

# img = [] # some array of images
# frames = [] # for storing the generated images
# fig = plt.figure()
# for i in range(0,x.shape[4]):
#     ## For L2 Distance
#     frames.append([plt.imshow(pred_np[0,:,:,i], cmap=cm.Greys_r,animated=True)])
# ani = animation.ArtistAnimation(fig, frames, interval=120, blit=True, repeat_delay=1000)
# ani.save('movie.mp4')


#### 3D convolution

##### Data Loading

In [ ]:
# Takes approximately 10 minutes 
x,y = dl.data_loader_3D('/content/data_new','/content/data_new',device)

## Printing the shape of specifid data
print('\nShape of MRI images - >');     dl.print_data_shape(x); 
print('\nShape of ASL images - >');     dl.print_data_shape(y);

## Printing the Stats of the data
print('\nStats for MRI data(X) - >');   dl.tensor_stats(x); 
print('\nStats for ASL data(X) - >');   dl.tensor_stats(y);

## Printing the Dimension of X and Y  
## Dimension refer -> Number x Length x Breadth x Height x Channel
print('\nDimensions of X(MRI Data) ->',end=' '); dl.print_data_dimension(x)
print('\nDimensions of Y(ASL Data) ->',end=' '); dl.print_data_dimension(y)

train_x , train_y , test_x , test_y = dl.data_split(x,y,0.2)

print('\nDimensions of Train Test Data (MRI) ->',end=' ');      dl.print_data_dimension(train_x) ; dl.print_data_dimension(test_x);
print('\nDimensions of Train Test Data (ASL Data) ->',end=' '); dl.print_data_dimension(train_y) ; dl.print_data_dimension(test_y);

##### Model Training


In [ ]:
train_loss ,test_loss, predictions = model.train_conv_3D(1250,4,train_x,train_y,test_x,test_y,4)
print('Test Loss ',test_loss); print('Predictions shape', predictions.shape)

##### Results and Video

In [ ]:
pd.predictions_3D(train_y,predictions,0)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
def plot_residuals_3D(y_target,y_predicted,sample_number):
    y_p = y_predicted[sample_number]; 
    y = y_target.cpu().detach().numpy(); y_p = y_p.cpu().detach().numpy()
    frames = [] # for storing the generated images
    fig = plt.figure(); 
    for i in range(0,y.shape[4]):
        ## Appendin the plots
        frames.append([plt.imshow((y[sample_number][0,:,:,i]- y_p[0,:,:,i])**2, cmap=cm.Greys_r,animated=True)]);
    ani = animation.ArtistAnimation(fig, frames, interval=120, blit=True, repeat_delay=1000)
    ani.save('Residual.mp4');
plot_residuals_3D(test_y,predictions,1)

from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Result.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)



In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Residual.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
del test_loss, predictions
